## Imports

In [1]:
# imports

import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

## Connecting to OpenAI

In [2]:
# Load environment variables in a file called .env

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

API key found and looks good so far!


In [3]:
openai = OpenAI()

## Call to Frontier model

In [4]:
# To give you a preview -- calling OpenAI with these messages is this easy. Any problems, head over to the Troubleshooting notebook.

message = "Hello, GPT! This is my first ever message to you! Hi!"
response = openai.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{
        "role":"user",
        "content":message
    }]
)

print(response.choices[0].message.content)

Hello! Welcome! I'm glad you decided to reach out. How can I assist you today?


## Create Website class

In [5]:
# A class to represent a Webpage
# If you're not familiar with Classes, check out the "Intermediate Python" notebook

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [6]:
# Let's try one out. Change the website and add print statements to follow along.

ed = Website("https://edwarddonner.com")
print(ed.title)
print(ed.text)

Home - Edward Donner
Home
Connect Four
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acquired in 2021
.
We work with groundbreaking, proprietary LLMs verticalized for talent, we’ve
patented
our matching model, and our award-winning platform has happy customers and tons of press coverage.
Connec

## Prompts (system, user)

In [7]:
# Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."

system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [8]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [9]:
print(user_prompt_for(ed))

You are looking at a website titled Home - Edward Donner
The contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.

Home
Connect Four
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acqui

## Messages

In [10]:
messages = [
    {
        "role": "system",
        "content": "You are a snarky assistant"
    },
    {
        "role": "user",
        "content": "What is 2 + 2?"
    }
]

In [11]:
# To give you a preview -- calling OpenAI with system and user messages:

response = openai.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages
)

print(response.choices[0].message.content)

Oh, we're starting off with the tough ones, huh? Well, brace yourself: 2 + 2 equals 4. Mind-blowing, I know! 


## Build useful messages using a function

In [12]:
# See how this function creates exactly the format above

def messages_for(website):
    return [
        {
            "role": "system",
            "content": system_prompt
        },
        {
            "role": "user",
            "content": user_prompt_for(website)
        }
    ]

In [13]:
# Try this out, and then try for a few more websites

messages_for(ed)

[{'role': 'system',
  'content': 'You are an assistant that analyzes the contents of a website and provides a short summary, ignoring text that might be navigation related. Respond in markdown.'},
 {'role': 'user',
  'content': 'You are looking at a website titled Home - Edward Donner\nThe contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.\n\nHome\nConnect Four\nOutsmart\nAn arena that pits LLMs against each other in a battle of diplomacy and deviousness\nAbout\nPosts\nWell, hi there.\nI’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (\nvery\namateur) and losing myself in\nHacker News\n, nodding my head sagely to things I only half understand.\nI’m the co-founder and CTO of\nNebula.io\n. We’re applying AI to a field where it can make a

## API for OpenAI

In [14]:
# And now: call the OpenAI API. You will get very familiar with this!

def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [15]:
summarize("https://edwarddonner.com")

"# Edward Donner's Website Summary\n\nEdward Donner is a software developer and co-founder/CTO of Nebula.io, which focuses on leveraging AI to aid talent discovery and management. He has a background in AI startups, previously founding untapt, which was acquired in 2021. Beyond coding and AI, he has interests in DJing and electronic music production.\n\n## Recent Announcements\n- **January 23, 2025**: LLM Workshop - Hands-on with Agents – resources available.\n- **December 21, 2024**: Welcome message for SuperDataScientists.\n- **November 13, 2024**: Resources shared for Mastering AI and LLM Engineering.\n- **October 16, 2024**: Resources for transitioning from Software Engineer to AI Data Scientist. \n\nThe site encourages connections and offers insights into the field of AI and LLMs."

In [16]:
# A function to display this nicely in the Jupyter output, using markdown

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [17]:
display_summary("https://edwarddonner.com")

# Website Summary - Edward Donner

Edward Donner's website showcases his passion for coding and experimenting with Large Language Models (LLMs). He is the co-founder and CTO of Nebula.io, a platform focused on helping individuals explore their potential through AI-enhanced talent management. The site also emphasizes his background as the founder of AI startup untapt, which was acquired in 2021.

## Key Features:
- **Connect Four**: A feature highlighting a unique interactive experience.
- **Outsmart**: An arena designed for LLM battles emphasizing strategy and diplomacy.

## Recent Announcements:
- **January 23, 2025**: Resources available for the "LLM Workshop – Hands-on with Agents."
- **December 21, 2024**: Announcement welcoming the "SuperDataScientists."
- **November 13, 2024**: Release of resources for "Mastering AI and LLM Engineering."
- **October 16, 2024**: Resources for transitioning "From Software Engineer to AI Data Scientist." 

The website invites visitors to connect with Ed through various social media platforms and email.

## Try more websites

In [18]:
display_summary("https://cnn.com")

# Summary of CNN Website

CNN is a leading news outlet that provides breaking news, analyses, and videos on various topics. The site covers a wide range of areas including:

- **National and International News**: Updates on pressing global affairs such as the Ukraine-Russia war and Israel-Hamas conflict.
- **Politics**: Coverage of US political dynamics, including presidential elections and significant policy changes.
- **Business**: Insights into financial markets, economic trends, and business news.
- **Health and Science**: Articles addressing health issues, advancements in science, and lifestyle tips.
- **Entertainment and Culture**: Reports on movies, celebrity news, and cultural events.
- **Sports**: Updates on various sports events and profiles of athletes.

## Recent Highlights
- **US Politics**: Tensions between Ukraine President Zelensky and former US President Trump surfaced, showcasing Russia's invasion impact.
- **Crisis Updates**: The ongoing unease in Gaza amid discussions for ceasefires between Israel and Hamas.
- **Health Discussions**: Debates surrounding the Social Security Administration's workforce cuts affecting service delivery.
- **Significant Events**: Reports on Pope Francis’ health and discussions of Ramadan celebrations.

Overall, CNN emphasizes a commitment to delivering timely news across diverse topics, alongside features aimed at engagement with their audience.

In [19]:
display_summary("https://anthropic.com")

# Anthropic Website Summary

Anthropic is an AI safety and research company based in San Francisco focused on developing reliable and beneficial AI systems. The company emphasizes safety in its AI products and research.

## Key Highlights:
- **Claude 3.7 Sonnet**: The latest AI model, described as the most intelligent yet, and the first hybrid reasoning model.
- **Claude Code**: A new tool aimed at coding, marketed as an "agentic tool."
- **Research**: Anthropic is dedicated to advancing AI safety, with notable publications such as "Constitutional AI: Harmlessness from AI Feedback" and "Core Views on AI Safety."

### Recent Announcements:
- **Claude 3.7 Sonnet Release**: Introduced as the most advanced model.
- **Claude Code Introduction**: Aimed at enhancing coding capabilities.

Anthropic’s interdisciplinary team combines expertise from machine learning, physics, policy, and product design to push the boundaries of AI safety and technology.

## pavelklos